> __《学术训练与职业伦理工作坊》“文本挖掘”专题__



- 罗晨
    - 武汉大学新闻与传播学院讲师
    - 邮箱：chenluo@whu.edu.cn


- 2022年10月28日09:50 - 12:15

- 语义网络分析（semantic network analysis, SMA）领域的重要研究者：[George Barnett](https://communication.ucdavis.edu/people/gbarnett)


- SMA适合用来[讲故事](https://journals.sagepub.com/doi/10.1177/08944393211012267)、进行[比较研究](https://www.tandfonline.com/doi/full/10.1080/17513050902759488)、进行[过程研究](https://ascelibrary.org/doi/full/10.1061/%28ASCE%29CO.1943-7862.0002041)


- SMA不适合作为一项独立的研究方法，除非你的研究问题非常重要/有趣、用于验证性研究、与其他方法进行交叉验证

### 1. 载入包

In [1]:
# os：和操作系统进行交互
# re：正则表达式
# jieba：中文分词组件
import os, re, jieba
# pandas：Python环境中（最）常用的数据分析组件
import pandas as pd
# jieba.posseg：jieba组件里的词性标注函数
import jieba.posseg as pseg
# opencc：中文繁简体转换组件
from opencc import OpenCC

### 2. 引入停用词

- [中文常用停用词表](https://github.com/goto456/stopwords)

In [2]:
for file in os.listdir("./data/stopwords_common/"):
    print(file)
    # 使用mac OS时，需要注意隐藏文件".DS_Store"

baidu_stopwords.txt
.DS_Store
scu_stopwords.txt
cn_stopwords.txt
hit_stopwords.txt


In [3]:
# 创建空列表，存储停用词
stopwords = list()
# 遍历存放停用词表文件夹中的文件
for file in os.listdir("./data/stopwords_common/"):
    # 排除隐藏文件
    if ".DS" not in file:
        with open("./data/stopwords_common/" + file, "r", encoding="utf-8") as f:
            for line in f.readlines():
                stopwords.append(line.strip())
print(len(stopwords))

3885


In [4]:
# 使用集合功能来对停用词表去重
stopwords = list(set(stopwords))
print(len(stopwords))

2313


### 3. 引入自定义词

In [6]:
userdict = list()
with open("./data/userdict.txt", "r", encoding="utf-8") as f:
    for line in f.readlines():
        userdict.append(line.strip())

userdict = list(set(userdict))
print(len(userdict))

with open("./data/userdict_new.txt", "w", encoding="utf-8") as t:
    for w in userdict:
        # n表示名词（noun），可以参考jieba文档来构建自定义词典
        t.write(str(w).strip() + " n" + "\n")

384


- 对于引入自定义词典前后的分词结果

In [7]:
# 引入前
for w in jieba.cut("华南海鲜市场在湖北"):
    print(w)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/z_/6b8f_9vs5cn5hx3_jlxf0jq80000gn/T/jieba.cache
Loading model cost 0.471 seconds.
Prefix dict has been built successfully.


华南
海鲜
市场
在
湖北


In [8]:
# 引入后
jieba.load_userdict("./data/userdict_new.txt")

for w in jieba.cut("华南海鲜市场在湖北"):
    print(w)

华南海鲜市场
在
湖北


### 4. 分词


- [词性表](https://gist.github.com/luw2007/6016931)


- 实用工具：[在线正则表达式验证](https://regex101.com/)

In [9]:
# 使用正则表达式定义目标抽取域
# 匹配中文
pattern_CN = re.compile(r"[\u4e00-\u9fa5]")
# 匹配词性：n - 名词、v - 动词、a - 形容词
pattern_init = re.compile(r"^[n|v|a]")

In [10]:
# 定义分词函数
def tokenize(text):
    # 用来存储分词结果的空列表
    kept_words = list()
    # 进行繁简体转换，并使用词性标注分词器
    for w, f in pseg.cut(OpenCC("t2s").convert(str(text).strip())):
        # 筛选条件：词语不在停用词表中 + 词语长度大于1 + 中文 + 词性符合指定词性（名/动/形）
        if (w not in stopwords) and \
        (len(w) > 1) and \
        (pattern_CN.search(w) != None) and \
        (pattern_init.search(f) != None):
            kept_words.append(w)
    # 最后一步：拼接列表
    return " ".join(kept_words)

### 5. 引入数据


- [Pandas文档](https://pandas.pydata.org/docs/reference/index.html#api)

In [11]:
corpus = pd.read_csv("./data/common_user.txt", 
                     header=None, 
                     index_col=None, 
                     sep="delimiter", 
                     # 不同解析器的优势不一样
                     engine="python")
corpus.head()

,0
0,北京公布2月18日新发新冠肺炎确诊病例活动过的小区或场所 http://t.cn/A6h0lufr
1,#全国性哀悼活动# [蜡烛]有些人的挚爱却永远留在了这个冬天。深切哀悼抗击新冠肺炎疫情斗争牺...
2,清明时节心情凝，探望长空思绪浓。忧怀自古英烈魂，更为新冠泪湿巾。天灾人祸举国哀，十亿龙人泪满...
3,截至4月26日18时，全球新冠肺炎疫情形势
4,#唐山确诊首例新型肺炎病例#坐标河北省唐山市迁安市沙河驿镇红庙子村，我们村开始给家家户户消毒...


In [12]:
# 查看数据维数
corpus.shape

(1000, 1)

In [13]:
# 更换列名
corpus.rename(columns={0: "content"}, inplace=True)
corpus.head()

,content
0,北京公布2月18日新发新冠肺炎确诊病例活动过的小区或场所 http://t.cn/A6h0lufr
1,#全国性哀悼活动# [蜡烛]有些人的挚爱却永远留在了这个冬天。深切哀悼抗击新冠肺炎疫情斗争牺...
2,清明时节心情凝，探望长空思绪浓。忧怀自古英烈魂，更为新冠泪湿巾。天灾人祸举国哀，十亿龙人泪满...
3,截至4月26日18时，全球新冠肺炎疫情形势
4,#唐山确诊首例新型肺炎病例#坐标河北省唐山市迁安市沙河驿镇红庙子村，我们村开始给家家户户消毒...


### 6. 正式分词

In [14]:
%%time
# 调用之前定义的分词函数
# lambda是匿名函数写法，传递的参数为形参
corpus["tokens"] = corpus["content"].apply(lambda x: tokenize(x))
corpus.head()

CPU times: user 4.76 s, sys: 93 ms, total: 4.85 s
Wall time: 4.85 s


,content,tokens
0,北京公布2月18日新发新冠肺炎确诊病例活动过的小区或场所 http://t.cn/A6h0lufr,北京 公布 新冠肺炎 确诊病例 活动 小区 场所
1,#全国性哀悼活动# [蜡烛]有些人的挚爱却永远留在了这个冬天。深切哀悼抗击新冠肺炎疫情斗争牺...,全国性 哀悼 活动 蜡烛 挚爱 留在 深切 哀悼 抗击 新冠肺炎 疫情 斗争 牺牲 烈士 逝...
2,清明时节心情凝，探望长空思绪浓。忧怀自古英烈魂，更为新冠泪湿巾。天灾人祸举国哀，十亿龙人泪满...,清明 时节 心情 探望 长空 思绪 忧怀 英烈 新冠 湿巾 举国 龙人 满怀 同胞 英魂 感...
3,截至4月26日18时，全球新冠肺炎疫情形势,全球 新冠肺炎 疫情 形势
4,#唐山确诊首例新型肺炎病例#坐标河北省唐山市迁安市沙河驿镇红庙子村，我们村开始给家家户户消毒...,唐山 确诊 首例 新型肺炎 病例 坐标 河北省 唐山市 迁安市 沙河 驿镇 红庙子 家家户户 消毒


In [15]:
# 核验空行，避免后续汇报出错
corpus[corpus["tokens"].apply(lambda x: len(x.strip().split())) == 0]

,content,tokens
145,『COVID-19: genetic network analysis provides ‘...,
249,"Coronavirus: 42 more cases in UK, taking total...",
449,Hanoi has had the first COVID-19 infection. t...,
546,Xi Focus: Xi calls for all-out global war agai...,
549,PHEIC,
578,Coronavirus: How California kept ahead of the ...,
589,Xi Focus: Xi calls for all-out global war agai...,


In [16]:
# 查看空行数量
corpus[corpus["tokens"].apply(lambda x: len(x.strip().split())) == 0].shape

(7, 2)

In [17]:
# 仅保留非空行
corpus = corpus[corpus["tokens"].apply(lambda x: len(x.strip().split())) > 0]
corpus.shape

(993, 2)

In [18]:
# 重新设定索引，避免后续出错
corpus.reset_index(drop=True, inplace=True)
corpus.tail()

,content,tokens
988,《独家｜上海研发新冠人源细胞疫苗，科学家已亲试第一针》自新冠肺炎疫情发生以来，我国的科研攻关...,独家 上海 新冠 人源 细胞 疫苗 科学家 新冠肺炎 疫情 发生 科研 攻关组 设立 疫苗 ...
989,#山西新增7例新型肺炎#只要B类控制住了，就能放心了。所以各位尽量别隐瞒自己的行动路线，没事...,山西 新增 新型肺炎 控制 放心 隐瞒 路线 出门 扎堆 显示 地图
990,『全国新型肺炎疫情实时动态 - 丁香园·丁香医生』http://t.cn/A6vBv3yL,全国 新型肺炎 疫情 动态 丁香 丁香 医生
991,#追星史大battle##眼泪及结膜分泌物存在新冠病毒#,追星 史大 眼泪 结膜 分泌物 新冠病毒
992,新冠期间比平时上班还忙，各种岗位轮换，还有线上教学各种事，我19年年假还没休呢，说好的三月带...,新冠 上班 岗位 轮换 教学 没休 说好 沙子 哪吒 委屈 算了 头箍 旅游 米虫 破坏 大...


### 7. SMA第一步：统计词频

In [19]:
# 需要去除检索词（避免超大集群出现），以及一些之前处理步骤中忽略的意义含量较低的词
dump = ["新冠", "新肺炎", "新型肺炎", "国际公共卫生紧急事件", "新冠肺炎", "新型冠状病毒肺炎", "肺炎", "不明原因肺炎", "不明肺炎", 
        "新冠疫情", "疫情", "新型冠状病毒", "肺炎疫情", "冠状", "冠状病毒", "新型冠状", "新冠病毒", "地图", "显示", "头条", 
        "文章", "微博", "视频", "真的"]

In [20]:
# numpy：常用的科学计算组件
import numpy as np
# collections：可以扩展Python内置的数据类型
from collections import defaultdict
# itertools：实现迭代功能的组件，可以快速计算排列组合
from itertools import combinations

In [21]:
# 构建defaultdict数据类型来统计词频
word_freq = defaultdict(int)

for line in corpus["tokens"]:
    for w in line.strip().split():
        if w not in dump:
            word_freq[w] += 1

with open("./data/word_freq.txt", "w", encoding="utf-8") as t:
    # 根据词频对词语进行排列（从高到低）
    for w, f in sorted(dict(word_freq).items(), key=lambda x: x[1], reverse=True):
        t.write(w + "," + str(f) + "\n")

### 8. SMA第二步：构建语义矩阵

In [22]:
# 提取前100个高频词
word_freq_100 = list()

word_freq = pd.read_csv("./data/word_freq.txt", 
                        header=None, 
                        index_col=None, 
                        sep=",", 
                        dtype={0: str, 1: np.float64})
word_freq.rename(columns={0: "词语", 1: "词频"}, 
                 inplace=True)

for w in word_freq["词语"][:100]:
    word_freq_100.append(w)

print(len(word_freq_100))
print(word_freq_100[:10])

100
['中国', '确诊', '美国', '武汉', '病例', '全国', '抗击', '新增', '患者', '加油']


In [23]:
# 存储两个词语之间的关系
edges = defaultdict(int)

for content in corpus["tokens"]:
    overlap = set(content.strip().split()) & set(word_freq_100)
    # 检验每一条微博的分词结果与前100个高频词之间是否存在交集
    if len(overlap) >= 2:
        idx = 0
        while idx <= len(content.strip().split()):
            # 检验我们设定的5-word窗口中是否存在高频词共现
            result = list(set(content.strip().split()[idx: idx + 5]) & overlap)
            # 存在共现的话……
            if len(result) >= 2:
            # 无向网络，不考虑词语的先后顺序
                result.sort()
                for pair in combinations(result, 2):
                    # 规避重复的共现统计
                    if content.strip().split()[idx: idx + 5][0] in pair:
                        edges[pair] += 1
            idx += 1

with open("./data/matrix.csv", "w", encoding="utf-8") as t:
    t.write("source, target, weight" + "\n")
    for dyad, weight in sorted(dict(edges).items(), key=lambda x: x[1], reverse=True):
        t.write(dyad[0] + "," + dyad[1] + "," + str(weight) + "\n")

### 9. SMA第三部：语义网络可视化

<img src="./corpus_network.png" width="600" align="middle">